In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = './images/'

# Получение путей файлов и меток
file_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir) if fname.endswith('.jpg')]
labels = [fname.split('_')[0] for fname in os.listdir(data_dir) if fname.endswith('.jpg')]

# Конвертируем метки в числовой формат
label_to_id = {label: idx for idx, label in enumerate(set(labels))}
ids = [label_to_id[label] for label in labels]

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(file_paths, ids, test_size=0.2, random_state=42)

# Создаем DataFrame для удобства работы с ImageDataGenerator
train_df = pd.DataFrame({
    'filename': X_train,
    'class': [label for id in y_train for label, idx in label_to_id.items() if idx == id]
})
test_df = pd.DataFrame({
    'filename': X_test,
    'class': [label for id in y_test for label, idx in label_to_id.items() if idx == id]
})

# Настройка генераторов данных
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='input',
    drop_remainder=True  # Обеспечивает, что все батчи будут иметь размер 32
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='class',
    target_size=(64, 64),
    batch_size=32,
    class_mode='input'
)

In [ ]:
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, Conv2DTranspose, LeakyReLU, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers

def build_generator(z_dim):
    model = Sequential()

    model.add(Input(shape=(z_dim,)))

    # Инициируем размер как 8x8
    model.add(Dense(256 * 8 * 8, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(Reshape((8, 8, 256)))

    # Первый увеличивающий слой до 16x16
    model.add(BatchNormalization())
    model.add(LeakyReLU(negative_slope=0.01))
    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)))

    # Второй увеличивающий слой до 32x32
    model.add(BatchNormalization())
    model.add(LeakyReLU(negative_slope=0.01))
    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)))

    # Третий увеличивающий слой до 64x64
    model.add(BatchNormalization())
    model.add(LeakyReLU(negative_slope=0.01))
    model.add(Conv2DTranspose(32, kernel_size=4, strides=2, padding='same', kernel_initializer=initializers.RandomNormal(stddev=0.02)))

    # Выходной слой с размером изображения 64x64 и трех цветовыми каналами
    model.add(Conv2DTranspose(3, kernel_size=4, strides=1, padding='same', activation='tanh'))

    return model

z_dim = 100
generator = build_generator(z_dim)

In [ ]:
from tensorflow.keras.layers import Input, Flatten, Conv2D, Dropout, LeakyReLU, Dense, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam, SGD

def build_discriminator(img_shape):
    model = Sequential()

    # Входной слой
    model.add(Input(shape=img_shape))

    # Первый сверточный слой
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', 
                     kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(negative_slope=0.01))  # Замена alpha на negative_slope
    model.add(Dropout(0.25))

    # Второй сверточный слой
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', 
                     kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(negative_slope=0.01))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())

    # Третий сверточный слой
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same', 
                     kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(negative_slope=0.01))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())

    # Выходной слой
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

img_shape = (64, 64, 3)
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.0002), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def build_gan(generator, discriminator):
    discriminator.trainable = False  # Замораживаем дискриминатор при обучении GAN

    # Входной слой для генератора (шумовой вектор z)
    z = Input(shape=(z_dim,))

    # Генерация изображения генератором
    img = generator(z)

    # Оценка дискриминатором сгенерированного изображения
    valid = discriminator(img)

    # Сборка модели GAN
    gan_model = Model(inputs=z, outputs=valid)
    gan_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

    return gan_model

gan = build_gan(generator, discriminator)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def train_gan(gan, generator, discriminator, train_generator, z_dim, epochs=1000, batch_size=128, sample_interval=100):
    for epoch in range(epochs):
        for imgs, _ in train_generator:
            current_batch_size = imgs.shape[0]  # Получаем размер текущего батча
            
            if current_batch_size != batch_size:
                continue  # Пропускаем батч, если его размер не соответствует ожидаемому

            real = np.ones((current_batch_size, 1))
            fake = np.zeros((current_batch_size, 1))

            z = np.random.normal(0, 1, (current_batch_size, z_dim))
            gen_imgs = generator.predict(z)

            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(imgs, real)
            d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            discriminator.trainable = False
            z = np.random.normal(0, 1, (current_batch_size, z_dim))
            g_loss = gan.train_on_batch(z, real)

            if (epoch + 1) % sample_interval == 0:
                print(f"Epoch {epoch + 1}: D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}% G loss: {g_loss:.4f}")
                sample_images(generator, z_dim, epoch)

            if current_batch_size < batch_size:
                break  # Выходим из цикла, если последний батч меньше ожидаемого

def sample_images(generator, z_dim, epoch, image_grid_rows=4, image_grid_columns=4):
    # Семплируем случайный шум
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # Генерируем изображения из шума
    gen_imgs = generator.predict(z)

    # Масштабируем изображения к диапазону [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Устанавливаем размеры сетки изображений
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(4, 4), sharey=True, sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, :], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("/kaggle/working/epoch_%d.png" % epoch)
    plt.close()

z_dim = 100
epochs = 10000
batch_size = 32
sample_interval = 1000

train_gan(gan, generator, discriminator, train_generator, z_dim, epochs, batch_size, sample_interval)